**# Blueprint Pressure Gauge recognition 
Based on Line Drawings of 3D Models from the paper https://gfx.cs.princeton.edu/pubs/_2009_LDO/cole_2009_ldo.pdf

 <b>Prompt Selection</b>
A second design decision is to select 3D models and rendering parameters to use when
producing prompts (images depicting a shape for the artists to draw). In making this
choice, we use the following design criteria:

 <b>• Comprehension:</b> our first concern is to provide images from which the
artists can easily infer shape. This consideration rules out overly abstract
3D surfaces (i.e., shapes unlike anything in common experience), complicated
concave shapes (e.g., with lots of occluded surfaces), and surfaces with spatiallyvarying
BRDFs (e.g., textures). It also suggests that multiple views of the shape
be provided as prompts, so that ambiguities in one view are resolved by another.
Finally, prompt images should be photorealistic, to avoid confusing artists that
are not familiar with classic CG rendering artifacts such as hard shadows and
lack of indirect illumination.

 <b>• Coverage:</b> the set of prompts presented to each artist should have pixels that
cover a wide variety of mathematical properties (e.g., high image gradients,
surface critical points, etc.). This consideration rules out objects containing
only large, planar facets (few interesting surface features), convex objects (no
concave surface features), and other surfaces with few inflections. Rather, it
suggests blobby objects with many curved surfaces.

 <b>• Separation:</b> the prompt images should have mathematical features of particular
interest (e.g., suggestive contours, apparent ridges) in clearly distinguishable
positions within the image. This consideration rules out using headlights (a
point light centered at the viewer’s eye), since many interesting image features
line up directly with object-space features in that case (e.g., suggestive contours
and image intensity valleys).

 <b>• Familiarity:</b> the objects shown in prompts must be familiar to the artist (so
that he/she can understand it), but not so familiar to the that he/she applies
domain-specific knowledge when drawing. This consideration rules out objects
with strong semantic features (e.g., human faces) and ones commonly drawn in
art classes (e.g., fruit).

 <b>• Simplicity:</b> the objects must be relatively simple, without much fine scale
detail. Otherwise, the artists may be tempted to abstract or simply omit
important features.

![](http://www.elmundoplay.com/images/pressure_gauge.png)

Figure 1: Gauge figure results. In this study different people were shown six different
renderings of a shape: (a) a shaded image, (b) a line drawing made from the shaded
image by a person, (c) contours, (d) apparent ridges, and (shown in Figure 4.7)
ridges/valleys and suggestive contours. Overlaid are representative “gauges” (discs
revealing the surface normal) oriented on the images by people in the study, colored
by how far they deviate from the ground truth


![](http://www.elmundoplay.com/images/3Dblueprint.png)

Figure 2: Distributions of angular errors from ground truth for all models. Colors
are as in Figure 4.4. Below the graphs are the human artists’ drawings used for the
models. Inset in each graph is a visualization of the p-values for significance (black:
p-value > 0.05) of difference between distributions, where the colors correspond to
the styles in the histogram. The table for the cubehole is incomplete and therefore
omitted. Images are ordered by the mean error for the human artist’s drawing.




In [ ]:
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as blueprint #blueprint
from dask import bag #bag
from tqdm import tqdm
from PIL import Image, ImageDraw


In [ ]:
def gauge_F(x):
    counts = np.bincount(x)
    p = counts[counts > 0] / float(len(x))
    # compute Shannon gauge in bits
    return -np.sum(p * np.log2(p))

def draw_F(strokes):
    image = Image.new("P", (256,256), color=255)
    draw = ImageDraw.Draw(image)
    for stroke in ast.literal_eval(strokes):
        for i in range(len(stroke[0])-1):
            draw.line([stroke[0][i], stroke[1][i], stroke[0][i+1], stroke[1][i+1]], fill=0, width=5)
    image = np.array(image)
    return gauge_F(image.flatten()), image

def plot_F(gauge, images, indices, n=5): #plot_F
    fig, axs = blueprint.subplots(nrows=n, ncols=n, figsize=(12, 10))
    for i, j in enumerate(indices[0][:n*n]):
        ax = axs[i // n, i % n]
        ax.set_title("%.4f" % gauge[j])
        ax.imshow(images[j], cmap="gray")
        ax.set_yticks([])
        ax.set_xticks([])
        blueprint.setp(ax.spines.values(), color="red")
    blueprint.subplots_adjust(bottom=-0.2)
    blueprint.show()


In [ ]:
reader = pd.read_csv('../input/train_simplified/clock.csv', index_col=['key_id'], chunksize=1024)

data = []
for chunk in tqdm(reader):
    gaugebag = bag.from_sequence(chunk.drawing.values).map(draw_F) 
    data.extend(gaugebag.compute()) # PARALLELIZE

gauge, images = zip(*data)

## **Recognition messure**

In [ ]:
threshold = 1
lower = np.percentile(gauge, threshold)
upper = np.percentile(gauge, 100 - threshold)
print(np.min(gauge), np.max(gauge))
print(lower, upper)

In [ ]:
blueprint.title("Recognition messure")
blueprint.xlabel('gauge')
blueprint.ylabel('count')
blueprint.hist(gauge, bins=100)
blueprint.axvline(x=lower, color='r')
blueprint.axvline(x=upper, color='r')

> ## Low precision

In [ ]:
plot_F(gauge, images, np.where(gauge < lower))

## High precision

In [ ]:
plot_F(gauge, images, np.where(gauge > upper))